In [1]:
"""
This notebook is used for model hyper-parameter searching.
Also, if this can also be used as a baseline training script.
"""
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib
import matplotlib.pyplot as plt
from pprint import pprint
from typing import Dict, List

In [2]:
import sys
sys.path.append("../")
# If this notebook file is not placed under in /notebook/ directory,
# adding directory "../" might not correly add the project directory.
# If adding "../" does not solve the importing problem, we need to setup 
# the directory mannually.
try:
    import constants
except ModuleNotFoundError:
    core_dir = input("Directory of core files >>> ")
    if not core_dir.endswith("/"):
        core_dir += "/"
    sys.path.append(core_dir)
    import constants

In [3]:
from core.tools.metrics import *
import core.tools.visualize as visualize
from core.tools.time_series import *
from core.tools.data_import import *
import core.tools.rnn_prepare as rnn_prepare
import core.tools.param_set_generator as param_set_generator

import core.models.stacked_lstm as stacked_lstm

import core.training.hps_methods as hps_methods

In [4]:
# data preparation phase.
pprint(constants.DATA_DIR)
choice = None
while choice is None or choice not in constants.DATA_DIR.keys():
    if choice is not None:
        print("Invalid data location received, try again...")
    choice = input("Select Dataset >>> ")

FILE_DIR = constants.DATA_DIR[choice]

print(f"Dataset chosen: {FILE_DIR}")

print("Avaiable configuration files found: ")
for cf in os.listdir("../hps_configs"):
    if cf.endswith("config.py"):
        print("\t" + cf)

config_name = input("Select config file >>> ")
if config_name.endswith(".py"):
    config_name = config_name[:-3]

exec(f"import hps_configs.{config_name} as config")

# print("Reading configuration file...")
# for att in dir(config):
#     if att.endswith("_config"):
#         print(f"\tLoading: {att}")
#         exec(f"globals().update(config.{att})")

{'a': '/Users/tianyudu/Documents/Academics/EconForecasting/AnnEconForecast/data/UNRATE.csv',
 'b': '/home/ec2-user/AnnEconForecast/data/UNRATE.csv',
 'c': '/home/ec2-user/AnnEconForecast/data/DEXCAUS.csv'}
Select Dataset >>> b
Dataset chosen: /home/ec2-user/AnnEconForecast/data/UNRATE.csv
Avaiable configuration files found: 
	ec2_config.py
	mac_config.py
	sample_config.py
Select config file >>> ec2_config


In [5]:
parameter_collection = param_set_generator.gen_hparam_set(config.main)

Total number of parameter sets generated: 54


In [6]:
pprint(parameter_collection[1])

{'LAGS': 6,
 'ORDER': 1,
 'PERIODS': 1,
 'TRAIN_RATIO': 0.8,
 'VAL_RATIO': 0.1,
 'clip_grad': None,
 'epochs': 150,
 'fig_path': '/home/ec2-user/ec2_hps/2018DEC20_01/model_figs/LAGS=6-epochs=150-num_neurons=(128, '
             '512)-learning_rate=0.03',
 'hparam_str': 'LAGS=6-epochs=150-num_neurons=(128, 512)-learning_rate=0.03',
 'learning_rate': 0.03,
 'model_path': '/home/ec2-user/ec2_hps/2018DEC20_01/saved_models/LAGS=6-epochs=150-num_neurons=(128, '
               '512)-learning_rate=0.03',
 'num_inputs': 1,
 'num_neurons': (128, 512),
 'num_outputs': 1,
 'num_time_steps': None,
 'report_periods': 10,
 'tensorboard_path': '/home/ec2-user/ec2_hps/2018DEC20_01/tensorboard/LAGS=6-epochs=150-num_neurons=(128, '
                     '512)-learning_rate=0.03'}


In [7]:
# Check something.

In [8]:
for (i, param) in enumerate(parameter_collection):
    print("================================================================")
    print(f"Executing hyper-parameter searching session [{i}/{len(parameter_collection) - 1}]...")
    print("Session Config:\n\t" + param["hparam_str"].replace("-", "\n\t"))
    start = datetime.now()
    hps_methods.individual_train(
        param=param,
        exec_core=stacked_lstm.exec_core,
        file_dir=FILE_DIR
    )
    print(f"Time taken for session [{i}]: {str(datetime.now() - start)}.")
print("Done.")

Executing hyper-parameter searching session [0/53]...
Session Config:
	LAGS=6
	epochs=150
	num_neurons=(128, 512)
	learning_rate=0.1
Dataset loaded.    
	Index type: datetime64[ns]    
	Data type: float64
StandardScaler applied, scaling based on the first 679 observations.
Total 842 observations generated.
Note: shape format: (num_obs, time_steps, num_inputs/outputs)
X shape = (842, 6, 1), y shape = (842, 1, 1)
Training and testing set generated,        
X_train shape: (672, 6, 1)        
y_train shape: (672, 1)        
X_test shape: (85, 6, 1)        
y_test shape: (85, 1)        
X_validation shape: (85, 6, 1)        
y_validation shape: (85, 1)
Resetting Tensorflow defalut graph...
Note: no gradient clipping is applied.            
If possible gradient exploding detected (e.g. nan loss), try use clip_grad.
Starting training session...
Training model...

Iteration [0], Training MSE 139644.0781250; Validation MSE 136888.9843750

Iteration [100], Training MSE 0.7608389; Validation MSE 

Note: no gradient clipping is applied.            
If possible gradient exploding detected (e.g. nan loss), try use clip_grad.
Starting training session...
Training model...

Iteration [0], Training MSE 136829.2187500; Validation MSE 150925.2500000

Iteration [100], Training MSE 1.0107262; Validation MSE 0.8031300

Iteration [200], Training MSE 0.9339539; Validation MSE 0.7571415
Saving the model...
Time taken for [300] epochs:  0:00:04.150779
Final result (validation set):
Loss Summary:
	mae=0.6436876058578491
	mse=0.7257505059242249
	rmse=0.8519099354743958
	mape=2.0611255168914795
Time taken for session [6]: 0:00:08.646553.
Executing hyper-parameter searching session [7/53]...
Session Config:
	LAGS=6
	epochs=300
	num_neurons=(128, 512)
	learning_rate=0.03
Dataset loaded.    
	Index type: datetime64[ns]    
	Data type: float64
StandardScaler applied, scaling based on the first 679 observations.
Total 842 observations generated.
Note: shape format: (num_obs, time_steps, num_inputs/out

Note: no gradient clipping is applied.            
If possible gradient exploding detected (e.g. nan loss), try use clip_grad.
Starting training session...
Training model...

Iteration [0], Training MSE 116183.9765625; Validation MSE 127763.7265625

Iteration [100], Training MSE 1.2290188; Validation MSE 0.8793129

Iteration [200], Training MSE 1.0137618; Validation MSE 0.7438293

Iteration [300], Training MSE 1.0077680; Validation MSE 0.7443495

Iteration [400], Training MSE 0.9890879; Validation MSE 0.7405894
Saving the model...
Time taken for [500] epochs:  0:00:06.312273
Final result (validation set):
Loss Summary:
	mae=0.6487454771995544
	mse=0.7401177287101746
	rmse=0.8603009581565857
	mape=2.493734836578369
Time taken for session [12]: 0:00:10.502757.
Executing hyper-parameter searching session [13/53]...
Session Config:
	LAGS=6
	epochs=500
	num_neurons=(128, 512)
	learning_rate=0.03
Dataset loaded.    
	Index type: datetime64[ns]    
	Data type: float64
StandardScaler applied, 

Time taken for session [17]: 0:00:12.740981.
Executing hyper-parameter searching session [18/53]...
Session Config:
	LAGS=12
	epochs=150
	num_neurons=(128, 512)
	learning_rate=0.1
Dataset loaded.    
	Index type: datetime64[ns]    
	Data type: float64
StandardScaler applied, scaling based on the first 679 observations.
Total 836 observations generated.
Note: shape format: (num_obs, time_steps, num_inputs/outputs)
X shape = (836, 12, 1), y shape = (836, 1, 1)
Training and testing set generated,        
X_train shape: (668, 12, 1)        
y_train shape: (668, 1)        
X_test shape: (84, 12, 1)        
y_test shape: (84, 1)        
X_validation shape: (84, 12, 1)        
y_validation shape: (84, 1)
Resetting Tensorflow defalut graph...
Note: no gradient clipping is applied.            
If possible gradient exploding detected (e.g. nan loss), try use clip_grad.
Starting training session...
Training model...

Iteration [0], Training MSE 322884.5000000; Validation MSE 344554.3750000

Itera

Note: no gradient clipping is applied.            
If possible gradient exploding detected (e.g. nan loss), try use clip_grad.
Starting training session...
Training model...

Iteration [0], Training MSE 10753.5078125; Validation MSE 8251.5849609

Iteration [100], Training MSE 0.6995137; Validation MSE 0.7837504

Iteration [200], Training MSE 0.5445993; Validation MSE 0.8070896
Saving the model...
Time taken for [300] epochs:  0:00:06.691067
Final result (validation set):
Loss Summary:
	mae=1.1079728603363037
	mse=1.7290136814117432
	rmse=1.3149195909500122
	mape=19.36020851135254
Time taken for session [24]: 0:00:10.817954.
Executing hyper-parameter searching session [25/53]...
Session Config:
	LAGS=12
	epochs=300
	num_neurons=(128, 512)
	learning_rate=0.03
Dataset loaded.    
	Index type: datetime64[ns]    
	Data type: float64
StandardScaler applied, scaling based on the first 679 observations.
Total 836 observations generated.
Note: shape format: (num_obs, time_steps, num_inputs/outp

Note: no gradient clipping is applied.            
If possible gradient exploding detected (e.g. nan loss), try use clip_grad.
Starting training session...
Training model...

Iteration [0], Training MSE 46693.6601562; Validation MSE 53391.0312500

Iteration [100], Training MSE 0.9132804; Validation MSE 0.7565892

Iteration [200], Training MSE 2.5131824; Validation MSE 3.0801876

Iteration [300], Training MSE 27.8492737; Validation MSE 28.4822769

Iteration [400], Training MSE 107.8427658; Validation MSE 103.4085846
Saving the model...
Time taken for [500] epochs:  0:00:10.569026
Final result (validation set):
Loss Summary:
	mae=2.3570499420166016
	mse=6.298251152038574
	rmse=2.509631633758545
	mape=44.47987365722656
Time taken for session [30]: 0:00:14.862270.
Executing hyper-parameter searching session [31/53]...
Session Config:
	LAGS=12
	epochs=500
	num_neurons=(128, 512)
	learning_rate=0.03
Dataset loaded.    
	Index type: datetime64[ns]    
	Data type: float64
StandardScaler applie

Saving the model...
Time taken for [500] epochs:  0:00:12.637622
Final result (validation set):
Loss Summary:
	mae=0.6104124188423157
	mse=0.653279721736908
	rmse=0.8082572221755981
	mape=5.3625807762146
Time taken for session [35]: 0:00:17.059538.
Executing hyper-parameter searching session [36/53]...
Session Config:
	LAGS=18
	epochs=150
	num_neurons=(128, 512)
	learning_rate=0.1
Dataset loaded.    
	Index type: datetime64[ns]    
	Data type: float64
StandardScaler applied, scaling based on the first 679 observations.
Total 830 observations generated.
Note: shape format: (num_obs, time_steps, num_inputs/outputs)
X shape = (830, 18, 1), y shape = (830, 1, 1)
Training and testing set generated,        
X_train shape: (664, 18, 1)        
y_train shape: (664, 1)        
X_test shape: (83, 18, 1)        
y_test shape: (83, 1)        
X_validation shape: (83, 18, 1)        
y_validation shape: (83, 1)
Resetting Tensorflow defalut graph...
Note: no gradient clipping is applied.            


Time taken for session [41]: 0:00:10.503442.
Executing hyper-parameter searching session [42/53]...
Session Config:
	LAGS=18
	epochs=300
	num_neurons=(128, 512)
	learning_rate=0.1
Dataset loaded.    
	Index type: datetime64[ns]    
	Data type: float64
StandardScaler applied, scaling based on the first 679 observations.
Total 830 observations generated.
Note: shape format: (num_obs, time_steps, num_inputs/outputs)
X shape = (830, 18, 1), y shape = (830, 1, 1)
Training and testing set generated,        
X_train shape: (664, 18, 1)        
y_train shape: (664, 1)        
X_test shape: (83, 18, 1)        
y_test shape: (83, 1)        
X_validation shape: (83, 18, 1)        
y_validation shape: (83, 1)
Resetting Tensorflow defalut graph...
Note: no gradient clipping is applied.            
If possible gradient exploding detected (e.g. nan loss), try use clip_grad.
Starting training session...
Training model...

Iteration [0], Training MSE 529604.3125000; Validation MSE 504690.5000000

Itera

Saving the model...
Time taken for [300] epochs:  0:00:10.933673
Final result (validation set):
Loss Summary:
	mae=0.6369342803955078
	mse=0.677377462387085
	rmse=0.8230294585227966
	mape=4.8433966636657715
Time taken for session [47]: 0:00:15.308156.
Executing hyper-parameter searching session [48/53]...
Session Config:
	LAGS=18
	epochs=500
	num_neurons=(128, 512)
	learning_rate=0.1
Dataset loaded.    
	Index type: datetime64[ns]    
	Data type: float64
StandardScaler applied, scaling based on the first 679 observations.
Total 830 observations generated.
Note: shape format: (num_obs, time_steps, num_inputs/outputs)
X shape = (830, 18, 1), y shape = (830, 1, 1)
Training and testing set generated,        
X_train shape: (664, 18, 1)        
y_train shape: (664, 1)        
X_test shape: (83, 18, 1)        
y_test shape: (83, 1)        
X_validation shape: (83, 18, 1)        
y_validation shape: (83, 1)
Resetting Tensorflow defalut graph...
Note: no gradient clipping is applied.          

Note: no gradient clipping is applied.            
If possible gradient exploding detected (e.g. nan loss), try use clip_grad.
Starting training session...
Training model...

Iteration [0], Training MSE 17296.8417969; Validation MSE 15281.0673828

Iteration [100], Training MSE 0.9977271; Validation MSE 0.9301739

Iteration [200], Training MSE 0.9264428; Validation MSE 0.8836356

Iteration [300], Training MSE 0.8995784; Validation MSE 0.8715892

Iteration [400], Training MSE 0.8808897; Validation MSE 0.8595878
Saving the model...
Time taken for [500] epochs:  0:00:17.437487
Final result (validation set):
Loss Summary:
	mae=1.7996138334274292
	mse=4.0027313232421875
	rmse=2.000682830810547
	mape=31.72930335998535
Time taken for session [53]: 0:00:21.902944.
Done.
